In [ ]:
!git clone https://github.com/lhldanh/Autoencoder-based-unsupervised-feature-learning-system.git
%cd Autoencoder-based-unsupervised-feature-learning-system/

In [ ]:
%mkdir -p build
%mkdir -p weights
%mkdir -p data
!wget https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz -O data/cifar-10-binary.tar.gz
!tar -xzvf data/cifar-10-binary.tar.gz -C data

In [ ]:
!nvcc -arch=sm_75 -o build/train_gpu_optimize src/train_gpu_optimize.cu src/optimize_kernel.cu src/cifar10_dataset.cpp -I include/

In [ ]:
%cd build/
!./train_gpu_optimize
%cd ..